In [1]:
docs=['python 데이터','데이터 분석','빅 데이터 분석','데이터 전처리']

vocab=list() # 단어사전 리스트
for doc in docs: # 문서
    for w in doc.split(): # 단어
        vocab.append(w) # 단어 추가

vocab=list(set(vocab)) # 중복 단어를 제거한 리스트
vocab.sort() # 오름차순 정렬
vocab

['python', '데이터', '분석', '빅', '전처리']

In [5]:
from math import log
N=len(docs) # 총 문서의 수

# TF(Term Frequency) : 단어의 빈도
# 특정 문서 d에서 특정 단어 t의 출현 빈도
def tf(t, d):
    return d.count(t)

# DF(Document Frequency) : 특정 단어가 등장한 문서의 수. 모든 문서에 자주 나오는 단어는 중요 단어가 아닐 수도..
# ex) 논문에서 abstract, 서론, 본론, 결론, 참고문헌 등 항상 나오는 단어들 - 논문 내용과는 무관

# IDF(Inverse Document Frequency) : DF에 반비례하는 수
# log( N/(df(t)+1) )
# 희귀 단어의 경우 분모가 매우 작아지게 되어 IDF값이 너무 커지게 되므로 로그를 취함
# 한번도 등장하지 않는 단어라면 분모가 0이 되므로 분모에 +1 처리
def idf(t):
    df=0
    for doc in docs:
        df+= t in doc
    return log(N/(df+1))

# TF와 IDF를 곱한 값
def tfidf(t,d):
    return tf(t,d)*idf(t)

In [3]:
import pandas as pd

result=[]

for i in range(N): # 문서 반복
    result.append([])
    d=docs[i]
    for j in range(len(vocab)):
        t=vocab[j]
        result[-1].append(tf(t,d)) # 1차원 배열의 끝에 추가

print(result)
df_tf=pd.DataFrame(result, columns=vocab)
df_tf # 단어의 출현 빈도

[[1, 1, 0, 0, 0], [0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 1]]


,python,데이터,분석,빅,전처리
0,1,1,0,0,0
1,0,1,1,0,0
2,0,1,1,1,0
3,0,1,0,0,1


In [6]:
# 각 단어의 IDF 계산
result=[]
for j in range(len(vocab)):
    t=vocab[j]
    result.append(idf(t))

df_idf=pd.DataFrame(result, index=vocab, columns=['IDF'])
df_idf

,IDF
python,0.693147
데이터,-0.223144
분석,0.287682
빅,0.693147
전처리,0.693147


In [7]:
# 모든 단어들의 TF-IDF 계산 : TF-IDF가 크면 중요도가 높고, 작으면 중요도가 낮음
result=[]
for i in range(N):
    result.append([])
    d=docs[i]
    for j in range(len(vocab)):
        t=vocab[j]
        result[-1].append(tfidf(t,d))

df_tfidf=pd.DataFrame(result,columns=vocab)
df_tfidf

,python,데이터,분석,빅,전처리
0,0.693147,-0.223144,0.000000,0.000000,0.000000
1,0.000000,-0.223144,0.287682,0.000000,0.000000
2,0.000000,-0.223144,0.287682,0.693147,0.000000
3,0.000000,-0.223144,0.000000,0.000000,0.693147


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
corpus=['You know I want your love', 'I like you', 'what should I do']
# DTM(Document Term Matrix, 문서 단어 행렬)
vector=CountVectorizer()
# 코퍼스로부터 각 단어의 빈도수 계산
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus=['python 데이터','데이터 분석','빅 데이터 분석','데이터 전처리']
tfidfv=TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)
# 사이킷런의 TF-IDF 계산 방식이 약간 달라서 차이가 나지만 일반적으로 많이 사용하는 함수
# TF-IDF에 L2 정규화를 적용하여 값을 조정함

[[0.88654763 0.46263733 0.         0.        ]
 [0.         0.55193942 0.83388421 0.        ]
 [0.         0.55193942 0.83388421 0.        ]
 [0.         0.46263733 0.         0.88654763]]
{'python': 0, '데이터': 1, '분석': 2, '전처리': 3}
